In [1]:
import math
import csv
import json

STAT_NAMES = [
    "caps", "garbage_time_caps", "hold", "ndps", "returns", "quick_returns", "nrts", "pups", 
    "keypops", "handoffs", "goodprevent", "resets", "badflaccids", "sparkedouts"
]

with open("smurfs.json") as f:
    smurfs = json.load(f)

def desmurf(player):
    player = player.replace("\\'", "'").replace("\\\\", "\\")
    if player in smurfs:
        return smurfs[player]
    return player

def row_to_dict(row):
    return {
        'match_id': int(row[0]),
        'timestamp': int(row[1]),
        'duration': int(row[2]),
        'cap_diff': int(row[3]),
        'garbage_time_cap_diff': int(row[4]),
        'players': [
            {
                'name': desmurf(row[j + 5]),
                'stats': {
                    stat_name: int(row[j * len(STAT_NAMES) + 13 + i])
                    for i, stat_name in enumerate(STAT_NAMES)
                }
            }
            for j in range(8)
        ]
    }

with open("matchups_with_stats.csv") as f:
    reader = csv.reader(f)
    next(reader)
    matches = [row_to_dict(row) for row in reader]

elo = {}
variance = {}
pred_errors = []
winner_win_prob = []

In [ ]:
STARTING_VARIANCE = 1.2
VARIANCE_TO_ADD_BACK = 0.002
BASE_VARIANCE = 4.5
RED_ADVANTAGE = 0.1
PLAYER_STAT_WEIGHTS_INITIAL = {
    "caps": 0.8,
    "garbage_time_caps": -0.3,
    "hold": 1.7 / 3600,
    "ndps": -0.5,
    "returns": 0.35,
    "quick_returns": 0.0,
    "nrts": 0.3,
    "pups": 0.3,
    "keypops": 0.0,
    "handoffs": 0.2,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_INITIAL = {
    "caps": 0,
    "garbage_time_caps": -0.3,
    "hold": 0.8 / 3600,
    "ndps": -0.0,
    "returns": 0.04,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
PLAYER_STAT_WEIGHTS_CONVERGED = {
    "caps": 0.05,
    "garbage_time_caps": -0.0,
    "hold": 0.0 / 3600,
    "ndps": -0.0,
    "returns": 0.0,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_CONVERGED = {
    "caps": 0,
    "garbage_time_caps": -0.0,
    "hold": 0.4 / 3600,
    "ndps": -0.0,
    "returns": 0.02,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": 0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": 0.0,
    "sparkedouts": 0.0
}
DIFF_MAPPING = [0, 0.2, 0.9, 2.0, 3.0, 4.1, -4.1, -3.0, -2.0, -0.9, -0.2]
NEW_PLAYER_ELO = -1.1
NEWNESS_THRESHOLD = 0.05
NEW_PLAYER_ELO_BOOST = 0.1
DIFF_WEIGHT_INITIAL = 0.6
DIFF_WEIGHT_CONVERGED = 0.7
GARBAGE_TIME_DISCOUNT = 0.4
RELATIVE_ELO_CORRECTION = 0.0  # Correction based on player's elo vs game average (negative = penalize high elo players)

In [3]:
def get_elo(player):
    if player not in elo:
        elo[player] = NEW_PLAYER_ELO
    return elo[player]

def get_variance(player):
    if player not in variance:
        variance[player] = STARTING_VARIANCE
    return variance[player]

def normalize_elos():
    known_elos = [elo[p] for p in elo if variance[p] <= 0.2]
    avg_elo = sum(known_elos) / max(10, len(known_elos))
    for p in elo:
        elo[p] -= avg_elo
        variance[p] = min(STARTING_VARIANCE, variance[p] + VARIANCE_TO_ADD_BACK)

In [ ]:
def judge_stats_initial(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def judge_stats_converged(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def update_rating(player, error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg):
    share_of_variance = variance[player] / total_variance
    newness = variance[player] / STARTING_VARIANCE
    error_weight = DIFF_WEIGHT_INITIAL * newness + DIFF_WEIGHT_CONVERGED * (1 - newness)
    stat_score = stat_score_initial * newness + stat_score_converged * (1 - newness)
    update = error_weight * error + stat_score
    update -= elo_vs_game_avg * RELATIVE_ELO_CORRECTION
    
    elo[player] += update * share_of_variance
    variance[player] *= 1 - share_of_variance
    if variance[player] > NEWNESS_THRESHOLD:
        elo[player] += NEW_PLAYER_ELO_BOOST * share_of_variance

def update_ratings(match):
    expected_score = RED_ADVANTAGE
    total_variance = BASE_VARIANCE
    
    for player in match['players'][:4]:
        expected_score += get_elo(player['name'])
        total_variance += get_variance(player['name'])
    for player in match['players'][4:]:
        expected_score -= get_elo(player['name'])
        total_variance += get_variance(player['name'])
    
    all_player_names = [p['name'] for p in match['players']]
    all_player_elos = [get_elo(name) for name in all_player_names]
    game_avg_elo = sum(all_player_elos) / len(all_player_elos)
    
    winner_win_prob.append(1 / (1 + math.exp(-expected_score / 2 if match['cap_diff'] > 0 else expected_score / 2)))
    pred_errors.append(match['cap_diff'] - min(max(expected_score, -5), 5))
    error = DIFF_MAPPING[match['cap_diff']] -\
        match['garbage_time_cap_diff'] * GARBAGE_TIME_DISCOUNT -\
        min(max(expected_score, DIFF_MAPPING[-5]), DIFF_MAPPING[5])
    red_stats_initial, blue_stats_initial = judge_stats_initial(match['players'])
    red_stats_converged, blue_stats_converged = judge_stats_converged(match['players'])
    for player, stat_score_initial, stat_score_converged in zip(match['players'][:4], red_stats_initial, red_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)
    for player, stat_score_initial, stat_score_converged in zip(match['players'][4:], blue_stats_initial, blue_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], -error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)

In [24]:
elo = {}
variance = {}
pred_errors = []
winner_win_prob = []
day = 0

for match in matches:
    new_day = round(match['timestamp'] / 86400)
    if new_day != day:
        normalize_elos()
    day = new_day
    update_ratings(match)

print(f"MAE: {sum([abs(e) for e in pred_errors]) / len(pred_errors):.4f}    MSE: {sum([e ** 2 for e in pred_errors]) / len(pred_errors):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob]) / len(winner_win_prob):.2%}    WWP: {sum(winner_win_prob) / len(winner_win_prob):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob]) / len(winner_win_prob)) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob]) / len(winner_win_prob)):.2%}")

MAE: 2.3853    MSE: 8.3046
COR: 59.73%    WWP: 52.77%
BRI: 51.55%    LOG: 51.61%


In [6]:
print(f"MAE: {sum([abs(e) for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}    MSE: {sum([e ** 2 for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:]):.2%}    WWP: {sum(winner_win_prob[10000:]) / len(winner_win_prob[10000:]):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])):.2%}")

MAE: 2.3804    MSE: 8.2596
COR: 58.81%    WWP: 52.22%
BRI: 51.23%    LOG: 51.26%


In [7]:
leaderboard = sorted([
    p for p in elo if variance[p] <= 0.3
], key=lambda x: elo[x], reverse=True)
for i, player in enumerate(leaderboard):
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}")

  1.  okthen         3.23 ± 0.54
  2.  CarrotCake     2.93 ± 0.42
  3.  DT             2.92 ± 0.59
  4.  toasty         2.78 ± 0.76
  5.  SluffAndRuff   2.76 ± 0.65
  6.  Alphachurro    2.56 ± 0.63
  7.  phreak         2.53 ± 0.56
  8.  OuchMyBalls    2.52 ± 0.35
  9.  meowza         2.49 ± 0.39
 10.  danp           2.45 ± 0.40
 11.  Ritual         2.38 ± 0.52
 12.  mex            2.33 ± 0.39
 13.  Xx360NoSwagx   2.30 ± 0.91
 14.  jig            2.29 ± 0.63
 15.  Suchit         2.28 ± 0.40
 16.  Shikari        2.26 ± 0.47
 17.  may_hem        2.21 ± 0.47
 18.  BALLDON'TLIE   2.19 ± 0.43
 19.  fender         2.19 ± 0.47
 20.  realtea        2.10 ± 0.44
 21.  Maelstrom      2.07 ± 0.41
 22.  ft             2.01 ± 0.59
 23.  Enervate       2.00 ± 0.42
 24.  Crippy         1.90 ± 0.62
 25.  Doris          1.88 ± 0.54
 26.  Messi          1.87 ± 0.63
 27.  Madoka         1.85 ± 0.95
 28.  Ball-E         1.81 ± 0.88
 29.  bbb            1.80 ± 0.52
 30.  eee            1.80 ± 0.43
 31.  Ty  

In [371]:
import random

ranks = []
for _ in range(10000):
    simulated_elos = {p: elo[p] + variance[p] ** 0.5 * random.gauss(0, 1) for p in elo if variance[p] <= 0.3}
    sorted_players = sorted(simulated_elos.items(), key=lambda x: x[1], reverse=True)
    ranks.append({player: rank + 1 for rank, (player, _) in enumerate(sorted_players)})

for i, player in enumerate(leaderboard[:]):
    rank_ci = sorted([r[player] for r in ranks])[500:-500]
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}  ({min(rank_ci)}-{max(rank_ci)})")

  1.  okthen         3.29 ± 0.54  (1-5)
  2.  CarrotCake     3.00 ± 0.42  (1-8)
  3.  DT             2.98 ± 0.59  (1-12)
  4.  toasty         2.83 ± 0.76  (1-21)
  5.  SluffAndRuff   2.81 ± 0.65  (1-18)
  6.  Alphachurro    2.61 ± 0.63  (2-25)
  7.  phreak         2.58 ± 0.56  (3-23)
  8.  OuchMyBalls    2.58 ± 0.35  (4-18)
  9.  meowza         2.54 ± 0.39  (4-20)
 10.  danp           2.51 ± 0.40  (5-22)
 11.  Ritual         2.42 ± 0.52  (5-29)
 12.  mex            2.38 ± 0.39  (7-26)
 13.  jig            2.34 ± 0.63  (5-38)
 14.  Xx360NoSwagx   2.34 ± 0.91  (3-54)
 15.  Suchit         2.33 ± 0.40  (7-29)
 16.  Shikari        2.30 ± 0.47  (7-33)
 17.  may_hem        2.26 ± 0.47  (8-35)
 18.  BALLDON'TLIE   2.25 ± 0.43  (8-34)
 19.  fender         2.24 ± 0.47  (8-36)
 20.  realtea        2.15 ± 0.44  (11-39)
 21.  Maelstrom      2.11 ± 0.41  (12-40)
 22.  Enervate       2.05 ± 0.42  (14-45)
 23.  ft             2.05 ± 0.59  (10-55)
 24.  Crippy         1.93 ± 0.62  (12-68)
 25.  Doris  